# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [45]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [46]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,marmarion,38.0484,24.3204,3.19,62,48,13.73,GR,1675651841
1,1,busselton,-33.6500,115.3333,25.82,49,0,5.19,AU,1675651842
2,2,puerto escondido,15.8500,-97.0667,24.69,72,0,2.06,MX,1675651842
3,3,iskateley,67.6803,53.1512,-19.45,99,36,3.62,RU,1675651842
4,4,odweyne,9.4092,45.0640,15.49,28,12,0.65,SO,1675651843


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [69]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    color = 'City', 
    s = 'Humidity',
    alpha = 0.65
)

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [48]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_city_df = city_data_df.loc[(city_data_df['Max Temp'] <= 30) & (city_data_df['Max Temp'] >= 20), :]

# Drop any rows with null values
ideal_city_df = ideal_city_df.dropna(how='any')

# Display sample data
ideal_city_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,1,busselton,-33.6500,115.3333,25.82,49,0,5.19,AU,1675651842
2,2,puerto escondido,15.8500,-97.0667,24.69,72,0,2.06,MX,1675651842
5,5,rikitea,-23.1203,-134.9692,26.04,76,2,6.82,PF,1675651843
6,6,avarua,-21.2078,-159.7750,26.03,89,100,5.66,CK,1675651844
8,8,nishihara,26.1842,127.7558,21.03,86,100,10.29,JP,1675651844


### Step 3: Create a new DataFrame called `hotel_df`.

In [49]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_city_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
1,busselton,AU,-33.6500,115.3333,49,
2,puerto escondido,MX,15.8500,-97.0667,72,
5,rikitea,PF,-23.1203,-134.9692,76,
6,avarua,CK,-21.2078,-159.7750,89,
8,nishihara,JP,26.1842,127.7558,86,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [50]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "limit":1,
    "apiKey":geoapify_key
}


# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lon = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lon},{lat},{radius}"
    params["bias"] = f"proximity:{lon},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
busselton - nearest hotel: Broadwater Beach Resort
puerto escondido - nearest hotel: Hotel Barlovento
rikitea - nearest hotel: Chez Bianca & Benoit
avarua - nearest hotel: Paradise Inn
nishihara - nearest hotel: ユインチホテル南城
batemans bay - nearest hotel: Esplanade Hotel
port alfred - nearest hotel: No hotel found
pondicherry - nearest hotel: Restaurant Paris Residency
moju - nearest hotel: No hotel found
victoria - nearest hotel: 四季酒店 Four Seasons Hotel
hithadhoo - nearest hotel: Pebbles Inn
butaritari - nearest hotel: No hotel found
isangel - nearest hotel: Tanna Lodge
cabo san lucas - nearest hotel: Comfort Rooms
buritizeiro - nearest hotel: Hotel Pousada Sertão Veredas
arraial do cabo - nearest hotel: No hotel found
cidreira - nearest hotel: Hotel Castelo
kapaa - nearest hotel: Pono Kai Resort
belmonte - nearest hotel: No hotel found
batu - nearest hotel: Batu Suki
bredasdorp - nearest hotel: Victoria Hotel
tautira - nearest hotel: No hotel found
banikoara - neare

zhuhai - nearest hotel: No hotel found
manhuacu - nearest hotel: Hotel Itália
banjar - nearest hotel: Villa Banjar Beach
makakilo city - nearest hotel: Embassy Suites by Hilton Oahu Kapolei
lorengau - nearest hotel: Seeadler Bay Hotel
sibolga - nearest hotel: Hotel Prima Indah
alta floresta - nearest hotel: Villas Hotel
sao miguel - nearest hotel: No hotel found
takoradi - nearest hotel: Hillcrest
ciudad bolivar - nearest hotel: Paoca
vao - nearest hotel: Nataiwatch
labuhan - nearest hotel: No hotel found
manyana - nearest hotel: No hotel found
saint-georges - nearest hotel: Deyna's City Inn
panama city - nearest hotel: Hostal Familiar Carmencita
moron - nearest hotel: La Esquina
muli - nearest hotel: Mulaku
paita - nearest hotel: Victor's
plettenberg bay - nearest hotel: Milkwood Manor
ikare - nearest hotel: No hotel found
buala - nearest hotel: No hotel found
la libertad - nearest hotel: Hotel Mar Caribe
tres arroyos - nearest hotel: Andrea Hotel
hobyo - nearest hotel: No hotel found

,City,Country,Lat,Lng,Humidity,Hotel Name
1,busselton,AU,-33.6500,115.3333,49,Broadwater Beach Resort
2,puerto escondido,MX,15.8500,-97.0667,72,Hotel Barlovento
5,rikitea,PF,-23.1203,-134.9692,76,Chez Bianca & Benoit
6,avarua,CK,-21.2078,-159.7750,89,Paradise Inn
8,nishihara,JP,26.1842,127.7558,86,ユインチホテル南城
...,...,...,...,...,...,...
523,buala,SB,-8.1450,159.5921,93,No hotel found
534,la libertad,EC,-2.2333,-80.9000,78,Hotel Mar Caribe
540,tres arroyos,AR,-38.3739,-60.2798,38,Andrea Hotel
541,hobyo,SO,5.3505,48.5268,76,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [68]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    hover_cols=["Hotel Name", "Country"],
    geo = True,
    tiles = "OSM",
    color = 'City', 
    s = 'Humidity',
    alpha = 0.65, 
    frame_width = 600, 
    frame_height = 500,
)

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)